# Find duplicate questions on StackOverflow by their embeddings

In this assignment you will learn how to calculate a similarity for pieces of text. Using this approach you will know how to find duplicate questions from [StackOverflow](https://stackoverflow.com).

### Libraries

In this task you will you will need the following libraries:
- [StarSpace](https://github.com/facebookresearch/StarSpace) — a general-purpose model for efficient learning of entity embeddings from Facebook
- [Gensim](https://radimrehurek.com/gensim/) — a tool for solving various NLP-related tasks (topic modeling, text representation, ...)
- [Numpy](http://www.numpy.org) — a package for scientific computing.
- [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
- [Nltk](http://www.nltk.org) — a platform to work with human language data.

### Data

The following cell will download all data required for this assignment into the folder `week3/data`.

In [34]:
import sys
sys.path.append("..")
from common.download_utils import download_week3_resources

#download_week3_resources()

### Grading
We will create a grader instace below and use it to collect your answers. Note that these outputs will be stored locally inside grader and will be uploaded to platform only after running submiting function in the last part of this assignment. If you want to make partial submission, you can run that cell any time you want.

In [83]:
from grader import Grader

In [84]:
grader = Grader()

## Word embedding

To solve the problem, you will use two different models of embeddings:

 - [Pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google which were trained on a part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases. `GoogleNews-vectors-negative300.bin.gz` will be downloaded in `download_week3_resources()`.
 - Representations using StarSpace on StackOverflow data sample. You will need to train them from scratch.

It's always easier to start with pre-trained embeddings. Unpack the pre-trained Goggle's vectors and upload them using the function [KeyedVectors.load_word2vec_format](https://radimrehurek.com/gensim/models/keyedvectors.html) from gensim library with the parameter *binary=True*. If the size of the embeddings is larger than the avaliable memory, you could load only a part of the embeddings by defining the parameter *limit* (recommended: 500000).

In [38]:
import gensim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import os
cwd = os.getcwd()

In [39]:
wv_embeddings = KeyedVectors.load_word2vec_format(datapath(cwd+'/GoogleNews-vectors-negative300.bin'), binary=True, limit=500000)  # C bin format

In [40]:
!ls

GoogleNews-vectors-negative300.bin     data	  week3-Embeddings.ipynb
GoogleNews-vectors-negative300.bin.gz  grader.py
__pycache__			       util.py


### How to work with Google's word2vec embeddings?

Once you have loaded the representations, make sure you can access them. First, you can check if the loaded embeddings contain a word:
    
    'word' in wv_embeddings
    
Second, to get the corresponding embedding you can use the square brackets:

    wv_embeddings['word']
 
### Checking that the embeddings are correct 
 
To prevent any errors during the first stage, we can check that the loaded embeddings are correct. You can call the function *check_embeddings*, implemented below, which runs 3 tests:
1. Find the most similar word for provided "positive" and "negative" words.
2. Find which word from the given list doesn’t go with the others.
3. Find the most similar word for the provided one.

In the right case the function will return the string *These embeddings look good*. Othervise, you need to validate the previous steps.

In [41]:
def check_embeddings(embeddings):
    error_text = "Something wrong with your embeddings ('%s test isn't correct)."
    most_similar = embeddings.most_similar(positive=['woman', 'king'], negative=['man'])
    if len(most_similar) < 1 or most_similar[0][0] != 'queen':
        return error_text % "Most similar"

    doesnt_match = embeddings.doesnt_match(['breakfast', 'cereal', 'dinner', 'lunch'])
    if doesnt_match != 'cereal':
        return error_text % "Doesn't match"
    
    most_similar_to_given = embeddings.most_similar_to_given('music', ['water', 'sound', 'backpack', 'mouse'])
    if most_similar_to_given != 'sound':
        return error_text % "Most similar to given"
    
    return "These embeddings look good."

In [42]:
print(check_embeddings(wv_embeddings))

These embeddings look good.


/usr/local/lib/python3.5/dist-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


## From word to text embeddings

**Task 1 (Question2Vec).** Usually, we have word-based embeddings, but for the task we need to create a representation for the whole question. It could be done in different ways. In our case we will use a **mean** of all word vectors in the question. Now you need to implement the function *question_to_vec*, which calculates the question representation described above. This function should work with the input text as is without any preprocessing.

Note that there could be words without the corresponding embeddings. In this case, you can just skip these words and don't take them into account during calculating the result. If the question doesn't contain any known word with embedding, the function should return a zero vector.

In [43]:
import numpy as np

In [44]:
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    words_embedding_score = np.zeros(dim)
    counter = 0
    for word in question.split():
        if word in embeddings:
            words_embedding_score += np.array(embeddings[word])
            counter += 1
    if counter != 0:
        words_embedding_score /= counter
    return words_embedding_score

To check the basic correctness of your implementation, run the function *question_to_vec_tests*.

In [45]:
def question_to_vec_tests():
    if (np.zeros(300) != question_to_vec('', wv_embeddings)).any():
        return "You need to return zero vector for empty question."
    if (np.zeros(300) != question_to_vec('thereisnosuchword', wv_embeddings)).any():
        return "You need to return zero vector for the question, which consists only unknown words."
    if (wv_embeddings['word'] != question_to_vec('word', wv_embeddings)).any():
        return "You need to check the corectness of your function."
    if ((wv_embeddings['I'] + wv_embeddings['am']) / 2 != question_to_vec('I am', wv_embeddings)).any():
        return "Your function should calculate a mean of word vectors."
    if (wv_embeddings['word'] != question_to_vec('thereisnosuchword word', wv_embeddings)).any():
        return "You should not consider words which embeddings are unknown."
    return "Basic tests are passed."

In [46]:
print(question_to_vec_tests())

Basic tests are passed.


You can submit embeddings for the questions from the file *test_embeddings.tsv* to earn the points. In this task you don't need to transform the text of a question somehow.

In [47]:
import nltk
nltk.download('stopwords')
from util import array_to_string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
question2vec_result = []
for question in open('data/test_embeddings.tsv'):
    question = question.strip()
    answer = question_to_vec(question, wv_embeddings)
    question2vec_result = np.append(question2vec_result, answer)

grader.submit_tag('Question2Vec', array_to_string(question2vec_result))

Current answer for task Question2Vec is: 0.019293891059027776
-0.028727213541666668
0.046056111653645836
0.08525933159722222
0.02430555555555...


Now we have a method to create a representation of any sentence and we are ready for the first evaluation. So, let's check how well our solution (Google's vectors + *question_to_vec*) will work.

## Evaluation of text similarity

We can imagine that if we use good embeddings, the cosine similarity between the duplicate sentences should be less than for the random ones. Overall, for each pair of duplicate sentences we can generate *R* random negative examples and find out the position of the correct duplicate.  

For example, we have the question *"Exceptions What really happens"* and we are sure that another question *"How does the catch keyword determine the type of exception that was thrown"* is a duplicate. But our model doesn't know it and tries to find out the best option also among questions like *"How Can I Make These Links Rotate in PHP"*, *"NSLog array description not memory address"* and *"PECL_HTTP not recognised php ubuntu"*. The goal of the model is to rank all these 4 questions (1 *positive* and *R* = 3 *negative*) in the way that the correct one is in the first place.

However, it is unnatural to count on that the best candidate will be always in the first place. So let us consider the place of the best candidate in the sorted list of candidates and formulate a metric based on it. We can fix some *K* — a reasonalble number of top-ranked elements and *N* — a number of queries (size of the sample).

### Hits@K

The first simple metric will be a number of correct hits for some *K*:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

where $q_i$ is the i-th query, $dup_i$ is its duplicate, $topK(q_i)$ is the top K elements of the ranked sentences provided by our model and the operation $[dup_i \in topK(q_i)]$ equals 1 if the condition is true and 0 otherwise (more details about this operation could be found [here](https://en.wikipedia.org/wiki/Iverson_bracket)).


### DCG@K
The second one is a simplified [DCG metric](https://en.wikipedia.org/wiki/Discounted_cumulative_gain):

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

where $rank_{dup_i}$ is a position of the duplicate in the sorted list of the nearest sentences for the query $q_i$. According to this metric, the model gets a higher reward for a higher position of the correct answer. If the answer does not appear in topK at all, the reward is zero. 

### Evaluation examples

Let's calculate the described metrics for the toy example introduced above. In this case $N$ = 1 and the correct candidate for $q_1$ is *"How does the catch keyword determine the type of exception that was thrown"*. Consider the following ranking of the candidates:
1. *"How Can I Make These Links Rotate in PHP"*
2. *"How does the catch keyword determine the type of exception that was thrown"*
3. *"NSLog array description not memory address"*
4. *"PECL_HTTP not recognised php ubuntu"*

Using the ranking above, calculate *Hits@K* metric for *K = 1, 2, 4*: 
 
- [K = 1] $\text{Hits@1} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top1(q_i)] = [dup_1 \in top1(q_1)] = 0$ because the correct answer doesn't appear in the *top1* list.
- [K = 2] $\text{Hits@2} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top2(q_i)] = [dup_1 \in top2(q_1)] = 1$ because $rank_{dup_1} = 2$.
- [K = 4] $\text{Hits@4} = \frac{1}{1}\sum_{i=1}^1 \, [dup_i \in top4(q_i)] = [dup_1 \in top4(q_1)] = 1$

Using the ranking above, calculate *DCG@K* metric for *K = 1, 2, 4*:

- [K = 1] $\text{DCG@1} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = \frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 1] = 0$ because the correct answer doesn't appear in the top1 list.
- [K = 2] $\text{DCG@2} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 2] = \frac{1}{\log_2{3}}$, because $rank_{dup_1} = 2$.
- [K = 4] $\text{DCG@4} = \frac{1}{1} \sum_{i=1}^1\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le 4] = \frac{1}{\log_2{3}}$.


**Tasks 2 and 3 (HitsCount and DCGScore).** Implement the functions *hits_count* and *dcg_score* as described above. Each function has two arguments: *dup_ranks* and *k*. *dup_ranks* is a list which contains *values of ranks* of duplicates. For example, *dup_ranks* is *[2]* for the example provided above.

In [49]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in Hits@k metric)

        result: return Hits@k value for current ranking
    """
    operation = np.array(dup_ranks) <= np.array([k])
    return np.average(operation)

Test your code on the tiny examples:

In [50]:
def test_hits():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1, 1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]
    
    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."

    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(hits_count(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [51]:
print(test_hits())

Basic test are passed.


In [52]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list of duplicates' ranks; one rank per question; 
                   length is a number of questions which we are looking for duplicates; 
                   rank is a number from 1 to len(candidates of the question); 
                   e.g. [2, 3] means that the first duplicate has the rank 2, the second one — 3.
        k: number of top-ranked elements (k in DCG@k metric)

        result: return DCG@k value for current ranking
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    operation = np.array(dup_ranks) <= np.array([k])
    return np.average((1/np.log2(1+np.array(dup_ranks)))*operation)
    

In [53]:
def test_dcg():
    # *Evaluation example*
    # answers — dup_i
    answers = ["How does the catch keyword determine the type of exception that was thrown"]
    
    # candidates_ranking — the ranked sentences provided by our model
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown",
                           "NSLog array description not memory address",
                           "PECL_HTTP not recognised php ubuntu"]]
    # dup_ranks — position of the dup_i in the list of ranks +1
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    
    # correct_answers — the expected values of the result for each k from 1 to 4
    correct_answers = [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function."
    
    # Other tests
    answers = ["How does the catch keyword determine the type of exception that was thrown", 
               "Convert Google results object (pure js) to Python object"]

    # The first test: both duplicates on the first position in ranked list
    candidates_ranking = [["How does the catch keyword determine the type of exception that was thrown",
                           "How Can I Make These Links Rotate in PHP"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [1, 1]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both duplicates on the first position in ranked list)."
        
    # The second test: one candidate on the first position, another — on the second
    candidates_ranking = [["How Can I Make These Links Rotate in PHP", 
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["Convert Google results object (pure js) to Python object",
                           "WPF- How to update the changes in list item of a list"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0.5, (1 + (1 / (np.log2(3)))) / 2]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: one candidate on the first position, another — on the second)."
        
    # The third test: both candidates on the second position
    candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                           "How does the catch keyword determine the type of exception that was thrown"], 
                          ["WPF- How to update the changes in list item of a list",
                           "Convert Google results object (pure js) to Python object"]]
    dup_ranks = [candidates_ranking[i].index(answers[i]) + 1 for i in range(len(answers))]
    correct_answers = [0, 1 / (np.log2(3))]
    for k, correct in enumerate(correct_answers, 1):
        if not np.isclose(dcg_score(dup_ranks, k), correct):
            return "Check the function (test: both candidates on the second position)."

    return "Basic test are passed."

In [54]:
print(test_dcg())

Basic test are passed.


Submit results of the functions *hits_count* and *dcg_score* for the following examples to earn the points.

In [55]:
test_examples = [
    [1],
    [1, 2],
    [2, 1],
    [1, 2, 3],
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [9, 5, 4, 2, 8, 10, 7, 6, 1, 3],
    [4, 3, 5, 1, 9, 10, 7, 8, 2, 6],
    [5, 1, 7, 6, 2, 3, 8, 9, 10, 4],
    [6, 3, 1, 4, 7, 2, 9, 8, 10, 5],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [86]:
hits_results = []
for example in test_examples:
    for k in range(len(example)):
        hits_results.append(hits_count(example, k + 1))
grader.submit_tag('HitsCount', array_to_string(hits_results))

Current answer for task HitsCount is: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1....


In [87]:
dcg_results = []
for example in test_examples:
    for k in range(len(example)):
        dcg_results.append(dcg_score(example, k + 1))
grader.submit_tag('DCGScore', array_to_string(dcg_results))

Current answer for task DCGScore is: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...


##  First solution: pre-trained embeddings

We will work with predefined train, validation and test corpora. All the files are tab-separated, but have a different format:
 - *train* corpus contains similar sentences at the same row.
 - *validation* corpus contains the following columns: *question*, *similar question*, *negative example 1*, *negative example 2*, ... 
 - *test* corpus contains the following columns: *question*, *example 1*, *example 2*, ...

Validation corpus will be used for the intermediate validation of models. The test data will be necessary for submitting the quality of your model in the system.

Now you should read *validation* corpus, located at `data/validation.tsv`. You will use it later to evaluate current solution.

In [58]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

In [59]:
validation = read_corpus('data/validation.tsv') ######### YOUR CODE HERE #############

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

We will use cosine distance to rank candidate questions which you need to implement in the function *rank_candidates*. The function should return a sorted list of pairs *(initial position in candidates list, candidate)*. Index of some pair corresponds to its rank (the first is the best). For example, if the list of candidates was *[a, b, c]* and the most similar is *c*, then *a* and *b*, the function should return a list *[(2, c), (0, a), (1, b)]*.

Pay attention, if you use the function *cosine_similarity* from *sklearn.metrics.pairwise* to calculate similarity because it works in a different way: most similar objects has greatest similarity. It's preferable to use a vectorized version of *cosine_similarity* function. Try to compute similarity at once and not use list comprehension. It should speed up your computations significantly.

In [61]:
def rank_candidates(question, candidates, embeddings, dim=300):
    """
        question: a string
        candidates: a list of strings (candidates) which we want to rank
        embeddings: some embeddings
        dim: dimension of the current embeddings
        
        result: a list of pairs (initial position in the list, question)
    """
    question_vec_score = question_to_vec(question, embeddings, dim)
    question_vec = np.full((1,dim), question_vec_score)
    candidates_vec = np.array([question_to_vec(candidate, embeddings, dim) for candidate in candidates])
    cosine_score = np.array(cosine_similarity(question_vec, candidates_vec)[0])
    merged_candidate_list = list(zip(cosine_score, range(len(candidates)), candidates))
    sorted_candidate_list  = sorted(merged_candidate_list, key=lambda x: x[0], reverse=True)
    result = [(index, candidate) for consine_score, index, candidate in sorted_candidate_list]
    return result

Test your code on the tiny examples:

In [62]:
def test_rank_candidates():
    questions = ['converting string to list', 'Sending array via Ajax fails']
    candidates = [['Convert Google results object (pure js) to Python object', 
                   'C# create cookie from string and send it',
                   'How to use jQuery AJAX for an outside domain?'], 
                  ['Getting all list items of an unordered list in PHP', 
                   'WPF- How to update the changes in list item of a list', 
                   'select2 not displaying search results']]
    results = [[(1, 'C# create cookie from string and send it'), 
                (0, 'Convert Google results object (pure js) to Python object'), 
                (2, 'How to use jQuery AJAX for an outside domain?')],
               [(0, 'Getting all list items of an unordered list in PHP'), 
                (2, 'select2 not displaying search results'), 
                (1, 'WPF- How to update the changes in list item of a list')]]
    for question, q_candidates, result in zip(questions, candidates, results):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, 300)
        if not np.all(ranks == result):
            return "Check the function."
    return "Basic tests are passed."

In [63]:
print(test_rank_candidates())

Basic tests are passed.


Now we can test the quality of the current approach. Run the next two cells to get the results. Pay attention that calculation of similarity between vectors takes time and this calculation is computed approximately in 10 minutes.

In [64]:
wv_ranking = []
for line in validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [65]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

DCG@   1: 0.212 | Hits@   1: 0.212
DCG@   5: 0.267 | Hits@   5: 0.315
DCG@  10: 0.282 | Hits@  10: 0.363
DCG@ 100: 0.320 | Hits@ 100: 0.552
DCG@ 500: 0.353 | Hits@ 500: 0.811
DCG@1000: 0.373 | Hits@1000: 1.000


If you did all the steps correctly, you should be frustrated by the received results. Let's try to understand why the quality is so low. First of all, when you work with some data it is necessary to have an idea how the data looks like. Print several questions from the data:

In [66]:
for line in validation[:3]:
    q, *examples = line
    print(q, *examples[:3])

How to print a binary heap tree without recursion? How do you best convert a recursive function to an iterative one? How can i use ng-model with directive in angular js flash: drawing and erasing
How to start PhoneStateListener programmatically? PhoneStateListener and service Java cast object[] to model WCF and What does this mean?
jQuery: Show a div2 when mousenter over div1 is over when hover on div1 depenting on if it is on div2 or not it should act differently How to run selenium in google app engine/cloud? Python Comparing two lists of strings for similarities


As you can see, we deal with the raw data. It means that we have many punctuation marks, special characters and unlowercased letters. In our case, it could lead to the situation where we can't find some embeddings, e.g. for the word "grid?". 

To solve this problem you should use the functions *text_prepare* from the previous assignments to prepare the data.

In [67]:
from util import text_prepare

Now transform all the questions from the validation set:

In [68]:
prepared_validation = []
for line in validation:
    prepared_validation.append([text_prepare(word) for word in line])

Let's evaluate the approach again after the preparation:

In [69]:
wv_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings)
    wv_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [70]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_prepared_ranking, k), 
                                              k, hits_count(wv_prepared_ranking, k)))

DCG@   1: 0.310 | Hits@   1: 0.310
DCG@   5: 0.380 | Hits@   5: 0.443
DCG@  10: 0.397 | Hits@  10: 0.494
DCG@ 100: 0.430 | Hits@ 100: 0.661
DCG@ 500: 0.453 | Hits@ 500: 0.835
DCG@1000: 0.470 | Hits@1000: 1.000


Now, prepare also train and test data, because you will need it in the future:

In [71]:
def prepare_file(in_, out_):
    out = open(out_, 'w')
    for line in open(in_, encoding='utf8'):
        line = line.strip().split('\t')
        new_line = [text_prepare(q) for q in line]
        print(*new_line, sep='\t', file=out)
    out.close()

In [72]:
prepare_file('data/train.tsv', 'data/prepared_train.tsv')
prepare_file('data/test.tsv', 'data/prepared_test.tsv')

**Task 4 (W2VTokenizedRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates to earn the points. The calculations should take about 3-5 minutes. Pay attention that the function *rank_candidates* returns a ranking, while in this case you should find a position in this ranking. Ranks should start with 1.

In [73]:
from util import matrix_to_string

In [88]:
w2v_ranks_results = []
prepared_test_data = 'data/prepared_test.tsv' ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, wv_embeddings, 300)
    ranked_candidates = [r[0] for r in ranks]
    w2v_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('W2VTokenizedRanks', matrix_to_string(w2v_ranks_results))

Current answer for task W2VTokenizedRanks is: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...


## Advanced solution: StarSpace embeddings

Now you are ready to train your own word embeddings! In particular, you need to train embeddings specially for our task of duplicates detection. Unfortunately, StarSpace cannot be run on Windows and we recommend to use provided
[docker container](https://github.com/hse-aml/natural-language-processing/blob/master/Docker-tutorial.md) or other alternatives. Don't delete results of this task because you will need it in the final project.

### How it works and what's the main difference with word2vec?
The main point in this section is that StarSpace can be trained specifically for some tasks. In contrast to word2vec model, which tries to train similar embeddings for words in similar contexts, StarSpace uses embeddings for the whole sentence (just as a sum of embeddings of words and phrases). Despite the fact that in both cases we get word embeddings as a result of the training, StarSpace embeddings are trained using some supervised data, e.g. a set of similar sentence pairs, and thus they can better suit the task.

In our case, StarSpace should use two types of sentence pairs for training: "positive" and "negative". "Positive" examples are extracted from the train sample (duplicates, high similarity) and the "negative" examples are generated randomly (low similarity assumed). 

### How to choose the best params for the model?
Normally, you would start with some default choice and then run extensive experiments to compare different strategies. However, we have some recommendations ready for you to save your time:
- Be careful with choosing the suitable training mode. In this task we want to explore texts similarity which corresponds to *trainMode = 3*.
- Use adagrad optimization (parameter *adagrad = true*).
- Set the length of phrase equal to 1 (parameter *ngrams*), because we need embeddings only for words.
- Don't use a large number of *epochs* (we think that 5 should be enough).
- Try dimension *dim* equal to 100.
- To compare embeddings usually *cosine* *similarity* is used.
- Set *minCount* greater than 1 (for example, 2) if you don't want to get embeddings for extremely rare words.
- Parameter *verbose = true* could show you the progress of the training process.
- Set parameter *fileFormat* equals *labelDoc*.
- Parameter *negSearchLimit* is responsible for a number of negative examples which is used during the training. We think that 10 will be enought for this task.
- To increase a speed of training we recommend to set *learning rate* to 0.05.

Train StarSpace embeddings for unigrams on the train dataset. You don't need to change the format of the input data. Just don't forget to use prepared version of the training data. 

If you follow the instruction, the training process will take about 1 hour. The size of the embeddings' dictionary should be approximately 100 000 (number of lines in the result file). If you got significantly more than this number, try to check all the instructions above.

In [75]:
######### TRAINING HAPPENING HERE #############
!starspace train -trainFile "data/prepared_train.tsv" -model starspace_embedding \
-trainMode 3 -adagrad true -ngrams 1 -epoch 5 -dim 100 -similarity cosine -minCount 2 \
-verbose true -fileFormat labelDoc -negSearchLimit 10 -lr 0.05

Arguments: 
lr: 0.05
dim: 100
epoch: 5
maxTrainTime: 8640000
saveEveryEpoch: 0
loss: hinge
margin: 0.05
similarity: cosine
maxNegSamples: 10
negSearchLimit: 10
thread: 10
minCount: 2
minCountLabel: 1
label: __label__
ngrams: 1
bucket: 2000000
adagrad: 1
trainMode: 3
fileFormat: labelDoc
normalizeText: 0
dropoutLHS: 0
dropoutRHS: 0
Start to initialize starspace model.
Build dict from input file : data/prepared_train.tsv
Read 12M words
Number of words in dictionary:  95058
Number of labels in dictionary: 0
Loading data from file : data/prepared_train.tsv
Total number of examples loaded : 999740
Initialized model weights. Model size :
matrix : 95058 100
Training epoch 0: 0.05 0.01


Epoch: 79.1%  lr: 0.042322  loss: 0.009911  eta: 0h13m  tot: 0h2m29s  (15.8%)%  lr: 0.049950  loss: 0.058007  eta: 0h15m  tot: 0h0m1s  (0.2%)3.1%  lr: 0.049750  loss: 0.035159  eta: 0h15m  tot: 0h0m6s  (0.6%)3.2%  lr: 0.049740  loss: 0.034745  eta: 0h15m  tot: 0h0m6s  (0.6%)5.2%  lr: 0.049510  loss: 0.028529  eta: 0h18m  tot: 0h0m11s  (1.0%)5.3%  lr: 0.049489  loss: 0.028202  eta: 0h18m  tot: 0h0m11s  (1.1%)5.6%  lr: 0.049459  loss: 0.027509  eta: 0h18m  tot: 0h0m12s  (1.1%)5.9%  lr: 0.049429  loss: 0.027010  eta: 0h18m  tot: 0h0m13s  (1.2%)%  lr: 0.049379  loss: 0.026249  eta: 0h18m  tot: 0h0m14s  (1.3%)6.7%  lr: 0.049359  loss: 0.025924  eta: 0h18m  tot: 0h0m14s  (1.3%)8.9%  lr: 0.049099  loss: 0.023236  eta: 0h17m  tot: 0h0m19s  (1.8%)12.2%  lr: 0.048779  loss: 0.020453  eta: 0h16m  tot: 0h0m25s  (2.4%)12.4%  lr: 0.048779  loss: 0.020277  eta: 0h16m  tot: 0h0m25s  (2.5%)12.7%  lr: 0.048749  loss: 0.020099  eta: 0h17m  tot: 0h0m26s  (2.5%)%  lr: 0.048619  loss: 0.019395  eta: 0h16m  

Epoch: 73.5%  lr: 0.032573  loss: 0.002698  eta: 0h9m  tot: 0h5m18s  (34.7%))%  lr: 0.039990  loss: 0.003533  eta: 0h4m  tot: 0h3m6s  (20.1%)0.8%  lr: 0.039950  loss: 0.003207  eta: 0h7m  tot: 0h3m6s  (20.2%)1.7%  lr: 0.039840  loss: 0.003331  eta: 0h9m  tot: 0h3m8s  (20.3%)2.1%  lr: 0.039780  loss: 0.003079  eta: 0h9m  tot: 0h3m9s  (20.4%)2.6%  lr: 0.039760  loss: 0.002947  eta: 0h9m  tot: 0h3m9s  (20.5%)3.6%  lr: 0.039660  loss: 0.002992  eta: 0h9m  tot: 0h3m11s  (20.7%)3.7%  lr: 0.039650  loss: 0.002961  eta: 0h10m  tot: 0h3m11s  (20.7%)%  lr: 0.039620  loss: 0.002974  eta: 0h10m  tot: 0h3m12s  (20.8%)4.3%  lr: 0.039600  loss: 0.002963  eta: 0h10m  tot: 0h3m12s  (20.9%)6.1%  lr: 0.039389  loss: 0.002755  eta: 0h10m  tot: 0h3m15s  (21.2%)7.6%  lr: 0.039269  loss: 0.002724  eta: 0h10m  tot: 0h3m18s  (21.5%)7.9%  lr: 0.039219  loss: 0.002784  eta: 0h10m  tot: 0h3m18s  (21.6%)9.0%  lr: 0.039129  loss: 0.002813  eta: 0h10m  tot: 0h3m20s  (21.8%)10.1%  lr: 0.039009  loss: 0.002845  eta: 0

Epoch: 100.0%  lr: 0.030000  loss: 0.002658  eta: 0h8m  tot: 0h6m3s  (40.0%)73.9%  lr: 0.032563  loss: 0.002698  eta: 0h9m  tot: 0h5m18s  (34.8%)74.0%  lr: 0.032553  loss: 0.002697  eta: 0h9m  tot: 0h5m19s  (34.8%)74.6%  lr: 0.032503  loss: 0.002701  eta: 0h9m  tot: 0h5m20s  (34.9%)74.9%  lr: 0.032483  loss: 0.002701  eta: 0h9m  tot: 0h5m20s  (35.0%)75.2%  lr: 0.032453  loss: 0.002697  eta: 0h9m  tot: 0h5m21s  (35.0%)75.7%  lr: 0.032413  loss: 0.002704  eta: 0h9m  tot: 0h5m22s  (35.1%)75.8%  lr: 0.032413  loss: 0.002704  eta: 0h9m  tot: 0h5m22s  (35.2%)76.2%  lr: 0.032362  loss: 0.002704  eta: 0h9m  tot: 0h5m23s  (35.2%)%  lr: 0.032212  loss: 0.002703  eta: 0h9m  tot: 0h5m25s  (35.5%)h5m26s  (35.6%)78.3%  lr: 0.032102  loss: 0.002706  eta: 0h9m  tot: 0h5m26s  (35.7%)78.4%  lr: 0.032082  loss: 0.002706  eta: 0h9m  tot: 0h5m26s  (35.7%)79.3%  lr: 0.032002  loss: 0.002712  eta: 0h9m  tot: 0h5m29s  (35.9%)80.4%  lr: 0.031932  loss: 0.002710  eta: 0h9m  tot: 0h5m30s  (36.1%)80.7%  lr: 0.031

Epoch: 80.7%  lr: 0.021832  loss: 0.001875  eta: 0h6m  tot: 0h8m24s  (56.1%)5%  lr: 0.029980  loss: 0.002045  eta: 0h6m  tot: 0h6m4s  (40.1%)1.9%  lr: 0.029820  loss: 0.002022  eta: 0h8m  tot: 0h6m7s  (40.4%)2.3%  lr: 0.029820  loss: 0.001930  eta: 0h8m  tot: 0h6m8s  (40.5%)3.1%  lr: 0.029760  loss: 0.001946  eta: 0h8m  tot: 0h6m9s  (40.6%)4.2%  lr: 0.029660  loss: 0.001935  eta: 0h9m  tot: 0h6m11s  (40.8%)4.8%  lr: 0.029630  loss: 0.001863  eta: 0h9m  tot: 0h6m12s  (41.0%)5.4%  lr: 0.029550  loss: 0.001895  eta: 0h8m  tot: 0h6m13s  (41.1%)5.6%  lr: 0.029540  loss: 0.001908  eta: 0h8m  tot: 0h6m14s  (41.1%)6.0%  lr: 0.029500  loss: 0.001880  eta: 0h8m  tot: 0h6m15s  (41.2%)6.3%  lr: 0.029489  loss: 0.001855  eta: 0h8m  tot: 0h6m15s  (41.3%)7.7%  lr: 0.029289  loss: 0.001812  eta: 0h8m  tot: 0h6m17s  (41.5%)9.2%  lr: 0.029119  loss: 0.001796  eta: 0h8m  tot: 0h6m20s  (41.8%)10.8%  lr: 0.028939  loss: 0.001827  eta: 0h8m  tot: 0h6m22s  (42.2%)12.1%  lr: 0.028759  loss: 0.001781  eta: 0h8

Epoch: 69.1%  lr: 0.012803  loss: 0.001570  eta: 0h3m  tot: 0h10m46s  (73.8%)3%  lr: 0.019980  loss: 0.001007  eta: 0h5m  tot: 0h8m55s  (60.1%)0.4%  lr: 0.019970  loss: 0.000893  eta: 0h5m  tot: 0h8m55s  (60.1%)1.3%  lr: 0.019880  loss: 0.001148  eta: 0h5m  tot: 0h8m57s  (60.3%)2.5%  lr: 0.019770  loss: 0.001289  eta: 0h5m  tot: 0h8m59s  (60.5%)2.7%  lr: 0.019760  loss: 0.001342  eta: 0h5m  tot: 0h8m59s  (60.5%)4.8%  lr: 0.019590  loss: 0.001386  eta: 0h5m  tot: 0h9m2s  (61.0%)4.9%  lr: 0.019590  loss: 0.001407  eta: 0h5m  tot: 0h9m2s  (61.0%)%  lr: 0.019580  loss: 0.001464  eta: 0h5m  tot: 0h9m3s  (61.0%)5.3%  lr: 0.019540  loss: 0.001475  eta: 0h5m  tot: 0h9m3s  (61.1%)5.9%  lr: 0.019469  loss: 0.001460  eta: 0h5m  tot: 0h9m4s  (61.2%)6.2%  lr: 0.019439  loss: 0.001458  eta: 0h5m  tot: 0h9m5s  (61.2%)6.6%  lr: 0.019419  loss: 0.001512  eta: 0h5m  tot: 0h9m5s  (61.3%)6.7%  lr: 0.019389  loss: 0.001510  eta: 0h5m  tot: 0h9m6s  (61.3%)7.0%  lr: 0.019339  loss: 0.001505  eta: 0h5m  tot: 

Epoch: 100.0%  lr: 0.010010  loss: 0.001563  eta: 0h2m  tot: 0h11m36s  (80.0%)0.5%  lr: 0.012703  loss: 0.001574  eta: 0h3m  tot: 0h10m48s  (74.1%)71.0%  lr: 0.012663  loss: 0.001571  eta: 0h3m  tot: 0h10m49s  (74.2%)74.5%  lr: 0.012382  loss: 0.001564  eta: 0h3m  tot: 0h10m55s  (74.9%)74.8%  lr: 0.012372  loss: 0.001565  eta: 0h3m  tot: 0h10m56s  (75.0%)74.9%  lr: 0.012372  loss: 0.001564  eta: 0h3m  tot: 0h10m56s  (75.0%)75.3%  lr: 0.012302  loss: 0.001562  eta: 0h3m  tot: 0h10m56s  (75.1%)75.4%  lr: 0.012292  loss: 0.001560  eta: 0h3m  tot: 0h10m57s  (75.1%)75.5%  lr: 0.012292  loss: 0.001560  eta: 0h3m  tot: 0h10m57s  (75.1%)75.7%  lr: 0.012292  loss: 0.001564  eta: 0h3m  tot: 0h10m57s  (75.1%)75.9%  lr: 0.012282  loss: 0.001565  eta: 0h3m  tot: 0h10m57s  (75.2%)77.1%  lr: 0.012182  loss: 0.001567  eta: 0h3m  tot: 0h10m59s  (75.4%)77.2%  lr: 0.012172  loss: 0.001568  eta: 0h3m  tot: 0h10m59s  (75.4%)77.4%  lr: 0.012172  loss: 0.001570  eta: 0h3m  tot: 0h11m0s  (75.5%)78.0%  lr: 0.0

Epoch: 87.8%  lr: 0.001472  loss: 0.001429  eta: <1min   tot: 0h13m56s  (97.6%).3%  lr: 0.009980  loss: 0.000938  eta: 0h1m  tot: 0h11m36s  (80.1%)0.4%  lr: 0.009940  loss: 0.001063  eta: 0h2m  tot: 0h11m36s  (80.1%)0.5%  lr: 0.009940  loss: 0.001074  eta: 0h2m  tot: 0h11m37s  (80.1%)%  lr: 0.009860  loss: 0.001321  eta: 0h2m  tot: 0h11m38s  (80.3%)2.2%  lr: 0.009810  loss: 0.001340  eta: 0h2m  tot: 0h11m39s  (80.4%)2.8%  lr: 0.009770  loss: 0.001360  eta: 0h2m  tot: 0h11m40s  (80.6%)3.3%  lr: 0.009730  loss: 0.001314  eta: 0h2m  tot: 0h11m41s  (80.7%)4.8%  lr: 0.009600  loss: 0.001351  eta: 0h2m  tot: 0h11m44s  (81.0%)4.9%  lr: 0.009600  loss: 0.001368  eta: 0h2m  tot: 0h11m44s  (81.0%)6.9%  lr: 0.009449  loss: 0.001351  eta: 0h2m  tot: 0h11m47s  (81.4%)7.7%  lr: 0.009379  loss: 0.001375  eta: 0h2m  tot: 0h11m48s  (81.5%)8.6%  lr: 0.009309  loss: 0.001388  eta: 0h2m  tot: 0h11m49s  (81.7%)  loss: 0.001388  eta: 0h2m  tot: 0h11m50s  (81.8%)8.9%  lr: 0.009299  loss: 0.001387  eta: 0h2m 

And now we can compare the new embeddings with the previous ones. You can find trained word vectors in the file *[model_file_name].tsv*. Upload the embeddings from StarSpace into a dict. 

In [76]:
######### YOUR CODE HERE #############
starspace_embeddings = {}  
for line in open('starspace_embedding.tsv', encoding='utf-8'):
    row = line.strip().split('\t')
    starspace_embeddings[row[0]] = np.array(row[1:], dtype=np.float32)

In [77]:
ss_prepared_ranking = []
for line in prepared_validation:
    q, *ex = line
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ss_prepared_ranking.append([r[0] for r in ranks].index(0) + 1)

In [78]:
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(ss_prepared_ranking, k), 
                                               k, hits_count(ss_prepared_ranking, k)))

DCG@   1: 0.518 | Hits@   1: 0.518
DCG@   5: 0.617 | Hits@   5: 0.701
DCG@  10: 0.635 | Hits@  10: 0.757
DCG@ 100: 0.666 | Hits@ 100: 0.906
DCG@ 500: 0.675 | Hits@ 500: 0.980
DCG@1000: 0.678 | Hits@1000: 1.000


Due to training for the particular task with the supervised data, you should expect to obtain a higher quality than for the previous approach. In additiion, despite the fact that StarSpace's trained vectors have a smaller dimension than word2vec's, it provides better results in this task.

**Task 5 (StarSpaceRanks).** For each question from prepared *test.tsv* submit the ranks of the candidates for trained representation.

In [89]:
starspace_ranks_results = []
prepared_test_data = prepared_test_data ######### YOUR CODE HERE #############
for line in open(prepared_test_data):
    q, *ex = line.strip().split('\t')
    ranks = rank_candidates(q, ex, starspace_embeddings, 100)
    ranked_candidates = [r[0] for r in ranks]
    starspace_ranks_results.append([ranked_candidates.index(i) + 1 for i in range(len(ranked_candidates))])
    
grader.submit_tag('StarSpaceRanks', matrix_to_string(starspace_ranks_results))

Current answer for task StarSpaceRanks is: 87	77	34	13	9	84	70	90	56	51	40	14	46	33	86	64	3	66	50	95	93	19	7	97	75	54	62	89	76	4	65	22	78	20	59...


Please, **don't remove** the file with these embeddings because you will need them in the final project.

### Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. <b>Note:</b> Token expires 30 minutes after generation.

In [90]:
STUDENT_EMAIL = 'er.bhavesh@live.com' # EMAIL 
STUDENT_TOKEN = 'jZ1MEAYyOaY3HGZB' # TOKEN 
grader.status()

You want to submit these parts:
Task Question2Vec: 0.019293891059027776
-0.028727213541666668
0.046056111653645836
0.08525933159722222
0.02430555555555...
Task HitsCount: 1.0
0.5
1.0
0.5
1.0
0.3333333333333333
0.6666666666666666
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1....
Task DCGScore: 1.0
0.5
0.8154648767857288
0.5
0.8154648767857288
0.3333333333333333
0.5436432511904858
0.7103099178...
Task W2VTokenizedRanks: 95	94	7	9	64	36	31	93	23	100	99	20	60	6	97	48	70	37	41	96	29	56	2	65	68	44	27	25	57	62	11	87	50	66	7...
Task StarSpaceRanks: 87	77	34	13	9	84	70	90	56	51	40	14	46	33	86	64	3	66	50	95	93	19	7	97	75	54	62	89	76	4	65	22	78	20	59...


If you want to submit these answers, run cell below

In [91]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
